# Text

NLP领域，recurrent neural network  
本章的目的是将text转换成RNN可以进行处理的内容，比如数字构成的tensor  
state-of-the-art - 最先进的

## reshaping data
有两个维度：character维度和word维度，但是方法一致：one-hot向量

In [1]:
with open('./data/jane-austen/1342-0.txt', encoding='utf-8') as f:
    text = f.read()

In [2]:
import torch
import os
import numpy as np

首先对文本进行分割，选择一行集中考虑

In [3]:
lines = text.split('\n')
line = lines[200]
line

'“Impossible, Mr. Bennet, impossible, when I am not acquainted with him'

对该行，创建一个tensor进行one-hot向量的编码

In [4]:
letter_tensor = torch.zeros(len(line), 128)
letter_tensor.shape

torch.Size([70, 128])

In [6]:
for i, letter in enumerate(line.lower().strip()):
    letter_index = ord(letter) if ord(letter) < 128 else 0
    letter_tensor[i][letter_index] = 1

`ord`函数会返回ASCII值

### 数据清洗
将去除标点符号  
`str.strip()`函数  
Python strip() 方法用于移除字符串头尾指定的字符（默认为空格或换行符）或字符序列

In [9]:
def clean_words(input_str):
    punctuation = '.,;:"!?_-”“'
    word_list = input_str.lower().replace('\n', ' ').split()
    word_list = [word.strip(punctuation) for word in word_list]
    return word_list

In [10]:
words_in_line = clean_words(line)
line, words_in_line

('“Impossible, Mr. Bennet, impossible, when I am not acquainted with him',
 ['impossible',
  'mr',
  'bennet',
  'impossible',
  'when',
  'i',
  'am',
  'not',
  'acquainted',
  'with',
  'him'])

### text分析
有了上面两部分内容和测试，可以对完整的text进行分析

In [12]:
word_list = sorted(set(clean_words(text)))
word2index_dict = {word: i for(i, word) in enumerate(word_list)}

len(word2index_dict), word2index_dict['impossible']

(7261, 3394)

word2index_dict是 word 词典，前面是索引值，后面是词  
word_tensor 是 word 的 one-hot 词向量构成的tensor
- 其内容的行数是**上面那句话**中的单词数量
- 其内容和列数是上面的单词再整个word词典中出现的位置

In [14]:
word_tensor = torch.zeros(len(words_in_line), len(word2index_dict))
for i, word in enumerate(words_in_line):
    word_index = word2index_dict[word]
    word_tensor[i][word_index] = 1
    print('{:2} {:4} {}'.format(i, word_index, word))
    
print(word_tensor.shape)

 0 3394 impossible
 1 4305 mr
 2  813 bennet
 3 3394 impossible
 4 7078 when
 5 3315 i
 6  415 am
 7 4436 not
 8  239 acquainted
 9 7148 with
10 3215 him
torch.Size([11, 7261])


### 词嵌入 text embeddings
从上面的体验中可以明确地感觉到一个问题，这个one-hot向量，有7261个数字，但是只有1个是1  
这样的tensor内容实际上很少，每增加一个新单词，又要增加一列，这样会极大增加训练网络的负担

下来需要使用浮点数的向量vector来表示单词words，这种方法是词嵌入 embedding  
大概是遍历一遍词典vocabulary，然后为每个单词word生成一组100个随机浮点数

这种方法听起来很有意思：  
比如要建立一些基础类别，猫科，水果，花，颜色。。。分别给一定的浮点数范围  
然后针对具体的某种物品，再进行分类，比如苹果：水果、红色等等【不过，这样岂不是再刚开始进行确定的时候会非常麻烦？】

实际中，基础类别 axes 不一定是有具体含义的，而是通过某个词周围的词 region、neighbor、nearby words等等来进行实现

通常使用神经网络来生成词嵌入，在此处不细讲